In [27]:
import itertools as it
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as it
from collections import defaultdict
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
path_common_source_trans = '/home/user/Desktop/courseProject/data/train_transaction.csv'
path_common_source_idn = '/home/user/Desktop/courseProject/data/train_identity.csv'
path_to_data_folder = '/home/user/Desktop/courseProject/data/'

In [4]:
clear_data = pd.read_csv(path_to_data_folder + '/clear_and_nan.csv')

In [22]:
art = []
nick = []
enj = []
i = 0
for col in clear_data.iloc[:,3:].columns:
    #print(col)
    i=i%3
    if i == 0:
        art.append(col)
    if i == 1:
        nick.append(col)
    if i == 2:
        enj.append(col)
    i+=1

In [23]:
print(len(art), len(nick), len(enj))

72 72 72


In [24]:
print(art)
print(nick)
print(enj)

['TransactionDT', 'card1', 'card4', 'addr1', 'P_emaildomain', 'C3', 'D1', 'D5', 'D15', 'M3', 'M6', 'M9', 'V3', 'V7', 'V10', 'V15', 'V23', 'V26', 'V32', 'V38', 'V42', 'V47', 'V53', 'V60', 'V65', 'V68', 'V75', 'V81', 'V86', 'V98', 'V104', 'V108', 'V111', 'V115', 'V118', 'V121', 'V124', 'V130', 'V136', 'V171', 'V174', 'V180', 'V185', 'V188', 'V194', 'V200', 'V207', 'V210', 'V217', 'V223', 'V227', 'V230', 'V240', 'V246', 'V250', 'V258', 'V262', 'V268', 'V281', 'V284', 'V287', 'V290', 'V303', 'V311', 'V314', 'id_01', 'id_06', 'id_13', 'id_17', 'id_28', 'id_35', 'id_38']
['TransactionAmt', 'card2', 'card5', 'addr2', 'R_emaildomain', 'C5', 'D3', 'D10', 'M1', 'M4', 'M7', 'V1', 'V4', 'V8', 'V12', 'V17', 'V24', 'V27', 'V35', 'V39', 'V44', 'V49', 'V55', 'V61', 'V66', 'V70', 'V77', 'V82', 'V87', 'V99', 'V105', 'V109', 'V112', 'V116', 'V119', 'V122', 'V125', 'V131', 'V169', 'V172', 'V175', 'V181', 'V186', 'V189', 'V195', 'V204', 'V208', 'V214', 'V220', 'V224', 'V228', 'V234', 'V241', 'V247', 'V252'

In [53]:
#y = clear_data['isFraud']
X = clear_data #.drop('isFraud', axis=1)

cat_dum = ['card4', 'card6','ProductCD', 'P_emaildomain', 'R_emaildomain', 'M1', 
           'M2', 'M3', 'M4','M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37'
          , 'id_38', 'DeviceType']
# code One Hot Encoding only cat_dum - cols
X = pd.get_dummies(X, drop_first=True, columns=cat_dum)

In [63]:
# code with mean
mean_dum = ['id_31', 'DeviceInfo', 'card1', 'card2',  'card3', 'card5', 'addr1', 'addr2']

def get_mean_encoding(df, group, target):
    """Group a Pandas DataFrame by a given column and return the mean encoding of the target variable for that grouping.
    
    Args:
        :param df: Pandas DataFrame.
        :param group: Column to group by. 
        :param target: Target variable column.
        
    Returns:
        Mean for the target variable across the group.
    
    Example: 
        df['mean_encoding_postcode_area'] = get_mean_encoding(df, 'postcode_area', 'attended')
    """
    
    mean_encoded = df.groupby(group)[target].mean()
    return df[group].map(mean_encoded)


X_red_mean = X.copy()
X_red_mean.drop(columns=mean_dum)
for col in mean_dum:
    t = get_mean_encoding(X, col, 'isFraud')
    print(t)
y = X['isFraud']
X = X.drop('isFraud', axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=123)

KeyError: 'id_31'

### Обучение модели и оценка качества

In [30]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(Xtrain, ytrain)

ValueError: could not convert string to float: 'W'